In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt



In [2]:
interior = ['kitchenapp', 'kitchenwall', 'kitchenfloor' ,
            'kitchenfurn', 'livingwall' ,'livingfloor', 'livingfurn',
 'bedwall', 'bedfloor' ,'bedfurn' ,'bathwall', 'bathfloor', 'bathfurn']


In [3]:
train_df = pd.read_csv('Training.csv', header = 0)
train_df.columns = train_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
train_df = train_df.astype(float)
train_df['interior'] = train_df[interior].sum(axis=1)
train_df.head()

,price,space_ft2,bedroom,bathroom,furnished,laundry,housing_type,parking,zipcode,exterior,...,livingfloor,livingfurn,bedwall,bedfloor,bedfurn,bathwall,bathfloor,bathfurn,studenthousing,interior
0,650.0,850.0,20.0,10.0,0.0,0.0,1.0,0.0,13790.0,2.0,...,3.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,0.0,34.0
1,650.0,600.0,10.0,10.0,0.0,3.0,1.0,4.0,13827.0,3.0,...,3.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,0.0,33.0
2,875.0,700.0,10.0,10.0,0.0,1.0,1.0,4.0,13045.0,2.0,...,3.0,2.0,3.0,2.0,1.0,3.0,3.0,2.0,0.0,31.0
3,1100.0,2120.0,30.0,15.0,0.0,2.0,1.0,2.0,13760.0,2.0,...,3.0,1.0,2.0,2.0,1.0,3.0,3.0,2.0,0.0,29.0
4,900.0,800.0,20.0,10.0,1.0,1.0,1.0,4.0,13905.0,2.0,...,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,0.0,33.0


In [4]:
test_df = pd.read_csv('Testing2_.csv', header =0)
test_df.columns = test_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
test_df = test_df.astype(float)
test_df['interior'] = test_df[interior].sum(axis=1)
test_df.head()

,space_ft2,bedroom,bathroom,furnished,laundry,housing_type,parking,zipcode,exterior,kitchenapp,...,livingwall,livingfloor,livingfurn,bedwall,bedfloor,bedfurn,bathwall,bathfloor,bathfurn,interior
0,850.0,20.0,10.0,0.0,0.0,1.0,0.0,13790.0,2.0,2.0,...,3.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,34.0
1,800.0,20.0,10.0,1.0,1.0,1.0,4.0,13905.0,2.0,2.0,...,3.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,33.0
2,2000.0,30.0,25.0,0.0,4.0,0.0,0.0,13905.0,2.0,2.0,...,3.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0,34.0
3,3000.0,80.0,30.0,0.0,3.0,0.0,4.0,13905.0,2.0,2.0,...,3.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,2.0,31.0
4,1100.0,10.0,10.0,0.0,1.0,1.0,7.0,13901.0,2.0,2.0,...,3.0,3.0,1.0,3.0,3.0,1.0,3.0,3.0,3.0,34.0


In [5]:
answer_df = pd.read_csv('Answer - Testing2 - Student.csv', header =0)
answer_df.columns = answer_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
answer_df = answer_df.astype(float)
answer_df.head()

,price,space_ft2,bedroom,bathroom,furnished,laundry,housing_type,parking,zipcode,exterior,...,kitchenfurn,livingwall,livingfloor,livingfurn,bedwall,bedfloor,bedfurn,bathwall,bathfloor,bathfurn
0,650.0,850.0,20.0,10.0,0.0,0.0,1.0,0.0,13790.0,2.0,...,2.0,3.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0
1,900.0,800.0,20.0,10.0,1.0,1.0,1.0,4.0,13905.0,2.0,...,2.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0
2,2400.0,2000.0,30.0,25.0,0.0,4.0,0.0,0.0,13905.0,2.0,...,3.0,3.0,3.0,2.0,3.0,3.0,2.0,3.0,3.0,2.0
3,3400.0,3000.0,80.0,30.0,0.0,3.0,0.0,4.0,13905.0,2.0,...,2.0,3.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,2.0
4,850.0,1100.0,10.0,10.0,0.0,1.0,1.0,7.0,13901.0,2.0,...,3.0,3.0,3.0,1.0,3.0,3.0,1.0,3.0,3.0,3.0


In [6]:
print(train_df.columns.values)

['price' 'space_ft2' 'bedroom' 'bathroom' 'furnished' 'laundry'
 'housing_type' 'parking' 'zipcode' 'exterior' 'kitchenapp' 'kitchenwall'
 'kitchenfloor' 'kitchenfurn' 'livingwall' 'livingfloor' 'livingfurn'
 'bedwall' 'bedfloor' 'bedfurn' 'bathwall' 'bathfloor' 'bathfurn'
 'studenthousing' 'interior']


In [7]:
#make predictions with
x = [ 'bedroom', 'space_ft2',
     'bathroom', 'interior', 'parking',
     'laundry', 'exterior','furnished',]

#bedroom 36
#space 11
#bathroom 4
#interior 4
#parking 4
#laundry 1
#exterior 1
#furnished 0

In [8]:
#predict
y = ['price']

In [9]:
knn = KNeighborsRegressor(algorithm='brute')

knn.fit(train_df[x], train_df[y])

predictions = knn.predict(test_df[x])

In [37]:
'answer : predictions\n'
for i in range(200):
    print([answer_df['price'][i] , predictions[i][0]])

[650.0, 760.0]
[900.0, 1019.0]
[2400.0, 1809.0]
[3400.0, 2348.0]
[850.0, 798.8]
[2750.0, 2340.0]
[900.0, 1164.4]
[1200.0, 1365.0]
[1400.0, 977.0]
[1700.0, 1560.0]
[1300.0, 1695.0]
[1500.0, 1500.0]
[525.0, 791.0]
[725.0, 801.0]
[1400.0, 1529.0]
[2000.0, 2000.0]
[1600.0, 1349.0]
[1500.0, 1435.0]
[650.0, 745.0]
[1100.0, 1170.0]
[3150.0, 3125.0]
[2250.0, 2320.0]
[1155.0, 1164.0]
[700.0, 870.8]
[2450.0, 2388.0]
[960.0, 1727.4]
[800.0, 760.0]
[4500.0, 3570.0]
[734.0, 1381.8]
[3325.0, 3125.0]
[750.0, 884.0]
[1425.0, 1099.6]
[2000.0, 2340.0]
[1875.0, 1349.0]
[2000.0, 1700.0]
[1155.0, 1164.0]
[1600.0, 1539.0]
[1300.0, 1555.0]
[1275.0, 1200.0]
[2450.0, 1895.0]
[2200.0, 2085.0]
[2375.0, 1800.0]
[550.0, 695.0]
[1500.0, 2230.0]
[1600.0, 1610.0]
[575.0, 907.8]
[1525.0, 1215.0]
[1155.0, 1164.0]
[2750.0, 2230.0]
[2900.0, 2100.0]
[1200.0, 1164.0]
[1155.0, 1164.0]
[775.0, 750.0]
[650.0, 1109.0]
[2200.0, 2228.0]
[600.0, 1009.0]
[1050.0, 1410.0]
[3325.0, 3125.0]
[1400.0, 1703.0]
[3325.0, 3125.0]
[675.0, 7

In [33]:
features_mse = mean_squared_error(answer_df['price'], predictions)
features_rmse = features_mse ** (1/2)
features_rmse

430.2372687250606